# Cleaning & Normalization To Do List  ✅

- ✅ Check if 'bookid' is unique - yes, but at end, I think we will drop it in favor of a serial ID
- ✅ Bookid is not unique figure out what to do - dropped duplicates. Difference was in price usually
- ✅ Pull ISBN from description field into ISBN field, if null or 99999999999 placeholder   
- NOTE ON ISBN INFO IN DESCRIPTION FIELD: I cannot now find a good way to eliinate duplicate ISBN information without eliminating some useful infrmation from the description field
- ✅ Split out extra ppl from author field, multiple authors
- ✅ Split out all list columns into separate tables
- ✅ Sort out cases where publish date is earlier than first publish date 
- ✅ Check dates - some mislabeled as 2000 instead of 1900
- ✅ Check for nulls, fill if possible?
- ✅ Strip extra spaces
- ✅ Eliminate duplicate entries
- ✅ Remove all non-numeric chars from numeric fields
- ✅ Standardize as many fields as possible
- ✅ Look up best way to normalize/handle ratings by stars
- ✅ Clean pages field of edition df
- ✅ Clean price field of edition df
- ✅ Create dfs
- ✅ Make sure you're not copying all null rows into dfs
- ✅ Set dtypes
- ✅ Create tables
- ✅ Create relationships
- ✅ Load data
- ✅ Create read user
- ✅ Filled null values with "missing" in creator table role column so that it could be included in the primary key


# Questions to ask stakeholders/users to make further improvements:
- Should we clean up the series_num field? There are severl instances where one title contains multiple joined instances from a series of smaller works. 
- There are 1200 editions with no star_ratings but with ratings and num_ratings values. Do we need to worry about those? Drop them?
- How important are the date fields? Should I clean them further?

# Analysis Notes

-  Regarding the "best books". The ratings seem almost irrelevant. Sales are what matter right? We don't have data on sales. What is a good proxy for sales?
Probably number of ratings, right?

- Probably can't just be the only metric though, bc then it would just be the most popular books. Need to combine with something else. Create a composite metric?

- There are 3-6 entries with duplicate ISBNs (depending on how you count) that are not '999999999' placeholders. I have not dropped them from the dfs, bc they have ratings data that might need to be aggregated across them. But I have flagged them with a boolean column `is_duplicate_isbn`.

Side bar: thinking about what to use as the "best" book metric. Is there a correlation between the number of ratings and the average rating?

In [ ]:
num_ratings_corr = df['numRatings'].corr(df['rating'])
print(f"Number of ratings and ratings correlation: {num_ratings_corr}")


# Normalization Notes:

1. First Normal Form (1NF)

- It only has atomic (indivisible) values. In other words, each cell should contain only one value, not a set of values or empty sets.
- Entries in a column are of the same kind. Each column should be of the same type (numeric, text, date, etc.).
- Each column in a table should represent a single attribute of the entity modelled by the table (e.g., a 'car' table might have separate columns for 'make', 'model', 'year', etc.)
- Order in which data is saved does not matter.

--

2. Second Normal Form (2NF)

- It is in 1NF.
- All non-key columns are fully dependent on the primary key. A non-key column must be functionally dependent on the entire set of primary key(s). There should be no partial dependency.
- In other words, if a column is dependent on only part of a multi-part primary key, then the table fails 2NF.

--

3. Third Normal Form (3NF)

- It is in 2NF.
- It has no transitive dependencies. A transitive dependency occurs when a non-key column is dependent on another non-key column, which is dependent on the primary key.
- Every non-key attribute must be functionally dependent on the primary key directly and not through some other non-key attributes.

